# Recipes Converter

A Simple Project that takes a cookbook/recipe book in pdf, converts each page to an image, use OpenAI to extract image contents, embed them and save in a FIASS vector store, and recieves queries on recipes, with responses retrieved from the vector store and passed to OpenAI chat completion for a well crafted response in markdown

## Setup

In [1]:
# Import the userdata module from Google Colab
from google.colab import userdata

# Retrieve OpenAI API key
api_key = userdata.get('openai_api')

In [2]:
# Mount the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Change directory to project folder
%cd /content/drive/MyDrive/AI\ Projects/RAG\ Course/6_RAG_With_OpenAI

/content/drive/MyDrive/AI Projects/RAG Course/6_RAG_With_OpenAI


## Install Libraries

In [4]:
# Install the pdf2image library for converting PDF files to images
!pip install pdf2image

# Install the poppler-utils package, required by pdf2image to work with PDF files
!apt-get install poppler-utils

# Install the openAI library
!pip install openai

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.10 [186 kB]
Fetched 186 kB in 0s (1,030 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126374 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.10_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.10) ...
Setting up poppler-utils (22.02.0-2ubuntu0.10) ...
Processing triggers for man-db (2.10.2-1) ...


## Import Libraries

In [5]:
from pdf2image import convert_from_path
from IPython.display import Markdown, display
from openai import OpenAI
import base64
import os
import json

## Perform OCR and Transform To Images

In [6]:
# Function to convert PDFs into images and stores the image paths
def pdf_to_images(pdf_path, output_folder):
  # Create the output folder if it doesn't exist
  if not os.path.exists(output_folder):
    os.makedirs(output_folder)

  # Convert PDF into images
  images = convert_from_path(pdf_path)

  # Save images and store their paths
  image_paths = []

  for i, image in enumerate(images):
    image_path = os.path.join(output_folder, f"page{i+1}.jpg") # Generate the image file path
    image.save(image_path, "JPEG") # Save the image as a JPEG file
    image_paths.append(image_path) # Append the image path to the list

  return image_paths # Return the list of image paths

In [7]:
# Define the path to the PDF and the output folder for images
pdf_path = "recipe_book.pdf"
output_folder = "images"

# Convert the PDF into images and store the image paths
image_paths = pdf_to_images(pdf_path, output_folder)

## Set Up OpenAI and The Model We Are Using

In [8]:
client = OpenAI(api_key=api_key)
model = "gpt-4o-mini"

### Test how things work on one image - Extract recipe info

In [9]:
# Read and encode one image
image_path = "images/page23.jpg" # Path to the image to be encoded

# Encode the image in base64 and decode to string
with open(image_path, "rb") as image_file:
  image_data = base64.b64encode(image_file.read()).decode('utf-8')
image_data

'...'

In [10]:
# Extract the recipe from the image

# Define the system prompt
system_prompt = """
Please analyze the content of this image and extract any related recipe information.
"""

# Call the OpenAI API use the chat completion method
response = client.chat.completions.create(
    model = model,
    messages = [
        # Provide the system prompt
        {"role": "system", "content": system_prompt},

        # The user message contains both the text and image URL / path
        {"role": "user", "content": [
            "This is the image from the recipe page.",
            {"type": "image_url",
             "image_url": {"url": f"data:image/jpeg;base64,{image_data}",
                           "detail": "low"}}
        ]}
    ]
)

# Retrieve the content
gpt_response = response.choices[0].message.content

# Display the GPT response as Markdown
display(Markdown(gpt_response))

It looks like you've shared a page from a recipe book. Based on the content generally found in such texts, here are the recipes mentioned:

### French Dressing
- **Ingredients:** 
  - 4 tablespoons olive oil
  - 1 tablespoon tarragon vinegar
  - ½ teaspoon white pepper
- **Instructions:** Whisk together olive oil and vinegar, add pepper, and serve.

### Mayonnaise Dressing
- **Ingredients:**
  - 2 hard-cooked yolks of eggs
  - ½ teaspoon salt
  - 1 teaspoon tarragon vinegar
  - ½ cup salad oil
  - 1 teaspoon lemon juice (optional)
- **Instructions:** Mash yolks, mix in salt and vinegar, add oil slowly while stirring, and incorporate lemon juice if desired.

### Hollandaise Sauce Supreme
- **Ingredients:**
  - ½ cup butter
  - 2 egg yolks
  - ¼ teaspoon salt
  - ¼ teaspoon white pepper
  - 2 tablespoons lemon juice
- **Instructions:** Melt butter, combine yolks and lemon juice over low heat, whisk until thick, and gently fold in butter.

### Rich Mayonnaise Dressing
- Similar ingredients and instructions as the standard mayonnaise dressing, with variations that may include additional enhancements or richness.

These recipes are classic dressings commonly used in various dishes. If you need more details or specific instructions from the text, please let me know!

## Let's Continue On All the Data - Extract all recipe information

In [11]:
# Define improved system prompt
system_prompt = """
Please analyze the content of this image and extract any related recipe information into structure components.
Specifically, extract the recipe title, list of ingredients, step by step instructions, cuisine type, dish type, any relevant tags or metadata.
The output must be formatted in a way suited for embedding in a Retrieval Augmented Generation (RAG) system.
"""

In [12]:
# Extract information about all of the images/recipes
extracted_recipes = []

for image_path in image_paths:
  print(f"Processing image {image_path}")

  # Reading and decoding the image
  with open(image_path, "rb") as image_file:
    image_data = base64.b64encode(image_file.read()).decode("utf-8") # Encode the image to base64 format

  # Call the API to extract the information
  response = client.chat.completions.create(
      model = model,
      messages = [
          # Provide system prompt for guidance
          {"role": "system", "content": system_prompt},

          # The user message contains both the text and image URL / path
          {"role": "user", "content": [
              "This is the image from the recipe page", # Context for the image
              {"type": "image_url",
              "image_url": {"url": f"data:image/jpeg;base64,{image_data}", # Provide the base64 image
                            "detail": "low"}}
          ]}
      ],
      temperature = 0, # Set the temperature to 0 for deterministic output
  )

  # Extract the content and store it
  gpt_response = response.choices[0].message.content # Get the response content
  extracted_recipes.append({"image_path": image_path, "recipe_info": gpt_response}) # Store the path and extracted info
  print(f"Extracted information for {image_path}:\n{gpt_response}\n") # Print the extracted information for review

Processing image images/page1.jpg
Extracted information for images/page1.jpg:
I can't analyze the content of the image directly. However, if you provide the text or details from the recipe, I can help you structure that information into components like the recipe title, ingredients, instructions, cuisine type, dish type, and any relevant tags.

Processing image images/page2.jpg
Extracted information for images/page2.jpg:
I'm unable to extract any recipe information from the image you provided, as it does not contain any visible recipe content. If you have another image or text with a recipe, feel free to share!

Processing image images/page3.jpg
Extracted information for images/page3.jpg:
I can't analyze the content of the image directly, but if you provide the text or details from the recipe, I can help you structure it into components like the title, ingredients, instructions, cuisine type, dish type, and any relevant tags.

Processing image images/page4.jpg
Extracted information for

In [13]:
# Filter out non-recipe content based on key recipe-related terms
filtered_recipes = []

for recipe in extracted_recipes:
  # Check if the extracted content contains any key recipe-related terms
  if any(keyword in recipe["recipe_info"].lower() for keyword in ["ingredients",
                                                                  "instructions",
                                                                  "recipe title"]):
     # If it does, add it to the filtered list
    filtered_recipes.append(recipe)

  # Print a message for non-recipe content
  else:
    print(f"Skipping recipe: {recipe['image_path']}")

Skipping recipe: images/page2.jpg
Skipping recipe: images/page4.jpg
Skipping recipe: images/page6.jpg
Skipping recipe: images/page7.jpg
Skipping recipe: images/page32.jpg
Skipping recipe: images/page51.jpg
Skipping recipe: images/page52.jpg


In [14]:
# Define the output file path
output_file = "recipes_from_recipe_book.json"

# Write the filtered list to a json file
with open(output_file, "w") as json_file:
  json.dump(filtered_recipes, json_file, indent = 2)

## Embeddings

In [15]:
# import libraries
import numpy as np

In [16]:
# Load the filtered recipes
with open("recipes_from_recipe_book.json", "r") as json_file:
  filtered_recipes = json.load(json_file)

In [17]:
# Generate embeddings for each recipe
recipe_texts = [recipe["recipe_info"] for recipe in filtered_recipes] # Extract the text content of each recipe

# Call the API to generate embeddings for the recipe texts
embedding_response = client.embeddings.create(
    input = recipe_texts,
    model = "text-embedding-3-large"
)

# Extract the embeddings
embeddings = [data.embedding for data in embedding_response.data]
embeddings

[[-0.021267471835017204,
  -0.03610227629542351,
  -0.019411349669098854,
  -0.017838604748249054,
  0.026368264108896255,
  -0.03783087804913521,
  -0.015245702117681503,
  -0.00046668711001984775,
  -0.016166679561138153,
  0.020601535215973854,
  0.019383011385798454,
  -0.03460037708282471,
  -0.011597218923270702,
  0.020417340099811554,
  0.029329558834433556,
  -0.03791589289903641,
  -0.014353063888847828,
  0.03179493919014931,
  -0.015274040400981903,
  -0.05157468095421791,
  0.014154699631035328,
  -0.002291814424097538,
  0.04168481007218361,
  0.0037405812181532383,
  -0.015089845284819603,
  -0.005763187073171139,
  -0.03771752864122391,
  0.0033225996885448694,
  0.014268050901591778,
  0.021168289706110954,
  0.014452246017754078,
  0.022344306111335754,
  0.015897469595074654,
  -0.029329558834433556,
  -0.028762802481651306,
  -0.028011852875351906,
  0.004675726406276226,
  0.04304502159357071,
  0.028238555416464806,
  0.026240743696689606,
  0.029003674164414406,


In [18]:
# Convert the embeddings to numpy array
embedding_matrix = np.array(embeddings)
embedding_matrix

array([[-0.02126747, -0.03610228, -0.01941135, ...,  0.00040182,
        -0.02706254,  0.00544085],
       [-0.01818025, -0.03156476, -0.02188606, ..., -0.00214174,
        -0.02582439,  0.00552965],
       [-0.01170185, -0.05251345, -0.02438443, ...,  0.00335825,
        -0.01866352,  0.0045953 ],
       ...,
       [-0.01063725, -0.02606478, -0.02589571, ..., -0.00131204,
        -0.02041506,  0.00165194],
       [-0.03386569, -0.03024324, -0.02926041, ..., -0.00075819,
        -0.01107796, -0.00516691],
       [-0.01172841, -0.05229815, -0.02415592, ...,  0.00331698,
        -0.01866729,  0.0046222 ]])

In [19]:
# Verify the embedding matrix
print(f"Generated embeddings for {len(filtered_recipes)} recipes.")
print(f"Each embedding is of size {len(embeddings[0])}")

Generated embeddings for 45 recipes.
Each embedding is of size 3072


## Retrieval System

In [20]:
# Install the faiss-cpu library
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 30.0 MB/s eta 0:00:00


In [21]:
# Import the faiss library
import faiss

In [22]:
# Print the embedding matrix shape
print(f"Embedding matrix shape: {embedding_matrix.shape}")

Embedding matrix shape: (45, 3072)


In [23]:
# Initialize the FAISS index for similarity search
index = faiss.IndexFlatL2(embedding_matrix.shape[1]) # Create a FAISS index with L2 distance metric
index.add(embedding_matrix) # Add the embeddings to the index

In [24]:
# Save the FAISS index to a file
faiss.write_index(index, "recipes_as_index.index")

In [25]:
# Save the metadata for each recipe
metadata = [{'recipe_info': recipe['recipe_info'], # Include recipe information
             'image_path': recipe['image_path']} for recipe in filtered_recipes] # Include image path

# Write metadata to a JSON file with indentation
with open("recipe_metadata.json", "w") as json_file:
  json.dump(metadata, json_file, indent = 2)

### Simple retriever demo

In [26]:
# Generate the embeddings for the query
query = "How to make bread?"
k = 5 # Number of top results to retrieve
query_embedding = client.embeddings.create(
    input = [query],
    model = "text-embedding-3-large"
).data[0].embedding
print(f"The query embedding is {query_embedding}\n")
query_vector = np.array(query_embedding).reshape(1, -1)  # Convert embedding to a 2D numpy array for FAISS
print(f"The query vector is {query_vector}\n")

# Search the FAISS index for the nearest neighbors
distances, indices = index.search(query_vector, min(k, len(metadata)))
print(f"The distances are {distances}\n")
print(f"The indices are {indices}\n")

# Store the indices and distances
stored_indices = indices[0].tolist()
stored_distances = distances[0].tolist()
print(f"The stored indices are {stored_indices}\n")
print(f"The stored distances are {stored_distances}\n")

# Print the metadata content for the top results
print("The metadata content is")
for i, dist in zip(stored_indices, stored_distances):
  if 0 <=i < len(metadata):
    print(f"Distance: {dist}, Metadata: {metadata[i]['recipe_info']}")

# Return the results
results = [(metadata[i]['recipe_info'], dist) for i, dist in zip(stored_indices, stored_distances) if 0 <= i < len(metadata)]
results # Output the results as a list of tuples containing recipe info and distance

The query embedding is [-0.019720381125807762, -0.028134725987911224, -0.022066915407776833, 0.016603518277406693, -0.04792621359229088, -0.04887430742383003, 0.038018617779016495, 0.017954552546143532, 0.0015480617294088006, 0.004168656188994646, -0.004832322709262371, -0.013889594934880733, -0.012917797081172466, -0.0015850967029109597, 0.038018617779016495, -0.0238090418279171, 0.017018308863043785, -0.016674624755978584, -0.0072233001701533794, 0.002310981974005699, -0.031856000423431396, 0.02015887387096882, 0.01501545775681734, -0.007821785286068916, 0.006275205407291651, 0.01784789189696312, 0.003410179866477847, -0.011229002848267555, -0.03960667923092842, 0.042285047471523285, -0.0036649806424975395, 0.023678677156567574, -0.034676581621170044, -0.02115437388420105, -0.015904298052191734, 0.011306035332381725, -0.01473102904856205, 0.005285630933940411, 0.00849730335175991, 0.019412249326705933, -0.01242004707455635, 0.007833636365830898, 0.00040108870598487556, -0.00825435388

[("Here’s the structured information extracted from the recipe page:\n\n### Recipe Title\nSally Lunn Hot Bread\n\n### Ingredients\n- 3 cups all-purpose flour\n- 2 tablespoons sugar\n- 1 teaspoon salt\n- 2 tablespoons butter\n- 1 cup milk\n- 2 eggs\n\n### Instructions\n1. Place sugar, eggs, milk, and butter in a bowl.\n2. Mix well and add flour and salt.\n3. Pour into a greased pan.\n4. Bake in a preheated oven at 400°F for 15 minutes.\n\n### Cuisine Type\nSouthern\n\n### Dish Type\nBread\n\n### Relevant Tags\n- Quick Bread\n- Southern Cuisine\n- Baking\n\n---\n\n### Recipe Title\nFour O'Clock Tea Scones\n\n### Ingredients\n- 2 cups pastry flour\n- 1 tablespoon sugar\n- 1 teaspoon baking powder\n- 1/2 teaspoon baking soda\n- 1/4 teaspoon salt\n- 1/4 cup butter\n- 1/2 cup buttermilk\n\n### Instructions\n1. Sift flour, sugar, baking powder, baking soda, and salt together.\n2. Cut in butter with a pastry blender.\n3. Add buttermilk gradually and mix until just combined.\n4. Turn onto a flo

In [27]:
# Define a function to query the embeddings
def query_embeddings(query, index, metadata, k = 5):
  # Generate the embeddings for the query
  query_embedding = client.embeddings.create(
      input = [query],
      model = "text-embedding-3-large"
  ).data[0].embedding
  print(f"The query embedding is {query_embedding}\n")
  query_vector = np.array(query_embedding).reshape(1, -1)
  print(f"The query vector is {query_vector}\n")

  # Search faiss index
  distances, indices = index.search(query_vector, min(k, len(metadata)))

  # Store the indices and distances
  stored_indices = indices[0].tolist()
  stored_distances = distances[0].tolist()
  print(f"The stored indices are {stored_indices}\n")
  print(f"The stored distances are {stored_distances}\n")

  # Return the results
  results = [(
      metadata[i]['recipe_info'], dist) for i, dist in zip(
          stored_indices, stored_distances) if 0 <= i < len(metadata)]
  return results

In [28]:
# Test the retrieval system
query = "chocolate query"
results = query_embeddings(query, index, metadata)
print(f"The results are {results}")

The query embedding is [-0.014626344665884972, -0.005924540106207132, -0.01682029664516449, -0.006895276717841625, -0.0328744500875473, 0.009994668886065483, 0.023471800610423088, 0.0519583486020565, -0.015261894091963768, 0.046490881592035294, 0.005841831676661968, -0.01802174560725689, -0.049799222499132156, 0.0076222410425543785, -0.025787638500332832, 0.014669875614345074, -0.013781847432255745, -0.015174832195043564, -0.020424645394086838, -0.03021036647260189, 0.02106890082359314, -0.01762126199901104, -0.005419583059847355, 0.02754628285765648, 0.023367326706647873, 0.04920720309019089, -0.011326710693538189, -0.00455767335370183, -0.038620512932538986, 0.012397568672895432, 0.01265004649758339, 0.018718238919973373, 0.027911940589547157, -0.021591270342469215, -0.02025052160024643, 0.009750896133482456, 0.01760384999215603, -0.038829460740089417, 0.015897443518042564, -0.009315588511526585, 0.02211363986134529, -0.03590419515967369, -0.021643508225679398, -0.0011677134316414595

In [29]:
# Combine the results into a single string
def combined_retrived_content(results):
  combined_content = "\n\n".join([result[0] for result in results]) # Join the recipe information with double newlines
  return combined_content

# Get the combined content from results
combined_content = combined_retrived_content(results)
print(f"The combined content is {combined_content}")

The combined content is I can't analyze the image directly, but I can help you structure the recipe information based on the text you provided. Here’s a structured format for the recipes mentioned:

### Recipe 1: Brown and White Cake
- **Cuisine Type:** American
- **Dish Type:** Cake
- **Ingredients:**
  - 1 cup butter
  - 2 cups sugar
  - 6 eggs, separated
  - 2 cups pastry flour
  - 1 teaspoon baking powder
  - 1 teaspoon cinnamon
  - 1 teaspoon clove
  - 1 cup milk
- **Instructions:**
  1. Cream butter and sugar, add egg yolks and mix well.
  2. Beat the egg whites until stiff and fold into the mixture.
  3. Divide the mixture into two parts; add spices and cocoa to one part.
  4. Pour into a well-greased loaf cake pan and bake at 350°F for 60 minutes.

### Recipe 2: Confederate Coffee Cake
- **Cuisine Type:** American
- **Dish Type:** Coffee Cake
- **Ingredients:**
  - 1 cup sugar
  - 1 cup butter
  - 2 cups flour
  - 1 teaspoon baking powder
  - 1 teaspoon baking soda
  - 1 cup bu

## Generative System

In [30]:
# Define the system prompt
system_prompt3 = f"""
You are highly experienced and expert chef specialized in providing cooking advice.
Your main task is to provide information precise and accurate on the combined content.
You answer diretly to the query using only information from the provided {combined_content}.
If you don't know the answer, just say that you don't know.
Your goal is to help the user and answer the {query}
"""

In [31]:
# Define function to retrieve a response from the API
def generate_response(query, combined_content, system_prompt):
  response = client.chat.completions.create(
      model = model,
      messages = [
          {"role": "system", "content": system_prompt}, # Provide system prompt for guidance
          {"role": "user", "content": query}, # Provide the query as user input
          {"role": "assistant", "content": combined_content} # Provide the combined content from the results
      ],
      temperature = 0, # Set temperature to 0 for deterministic output
  )
  return response

In [32]:
# Define a function to get the GPT response and display it in Markdown
def get_gpt_response():
  gpt_response = response.choices[0].message.content # Extract the response content from the API response
  return display(Markdown(gpt_response)) # Display the response as Markdown

In [33]:
# Get the results from the API
query = "How to make bread?"
combined_content = combined_retrived_content(results)
response = generate_response(query, combined_content, system_prompt3)

In [34]:
# Display the outcome
get_gpt_response()

Here’s a simple recipe for making bread:

### Basic Bread Recipe

#### Ingredients:
- 4 cups all-purpose flour
- 1 packet (2 ¼ teaspoons) active dry yeast
- 1 ½ cups warm water (about 110°F or 43°C)
- 2 tablespoons sugar
- 2 teaspoons salt
- 2 tablespoons olive oil (optional)

#### Instructions:
1. **Activate Yeast:**
   - In a small bowl, combine warm water and sugar. Sprinkle the yeast over the top and let it sit for about 5-10 minutes until it becomes frothy.

2. **Mix Ingredients:**
   - In a large mixing bowl, combine the flour and salt. Make a well in the center and add the yeast mixture and olive oil (if using).

3. **Knead the Dough:**
   - Mix until a dough forms. Transfer the dough to a floured surface and knead for about 8-10 minutes until smooth and elastic.

4. **First Rise:**
   - Place the dough in a greased bowl, cover it with a clean cloth or plastic wrap, and let it rise in a warm place for about 1-2 hours, or until it has doubled in size.

5. **Shape the Bread:**
   - Punch down the risen dough to release air. Shape it into a loaf or divide it into rolls. Place it in a greased loaf pan or on a baking sheet.

6. **Second Rise:**
   - Cover the shaped dough and let it rise again for about 30-60 minutes, until it has puffed up.

7. **Preheat Oven:**
   - Preheat your oven to 375°F (190°C).

8. **Bake:**
   - Bake the bread for 25-30 minutes, or until it is golden brown and sounds hollow when tapped on the bottom.

9. **Cool:**
   - Remove the bread from the oven and let it cool on a wire rack before slicing.

### Enjoy your homemade bread!

Feel free to ask if you have any questions or need further assistance!

In [35]:
# Get the results
query = "Get me the best chocolate cake recipe"
combined_content = combined_retrived_content(results)
response = generate_response(query, combined_content, system_prompt3)

# Display the outcome
get_gpt_response()

Here’s a delicious chocolate cake recipe that you can try:

### Recipe: Classic Chocolate Cake
- **Cuisine Type:** American
- **Dish Type:** Cake
- **Ingredients:**
  - 1 ¾ cups all-purpose flour
  - 1 ¾ cups granulated sugar
  - ¾ cup unsweetened cocoa powder
  - 1 ½ teaspoons baking powder
  - 1 ½ teaspoons baking soda
  - 1 teaspoon salt
  - 2 large eggs
  - 1 cup whole milk
  - ½ cup vegetable oil
  - 2 teaspoons vanilla extract
  - 1 cup boiling water

- **Instructions:**
  1. Preheat your oven to 350°F (175°C). Grease and flour two 9-inch round cake pans.
  2. In a large mixing bowl, combine the flour, sugar, cocoa powder, baking powder, baking soda, and salt.
  3. Add the eggs, milk, oil, and vanilla to the dry ingredients. Beat on medium speed for 2 minutes.
  4. Stir in the boiling water (the batter will be thin).
  5. Pour the batter evenly into the prepared cake pans.
  6. Bake for 30 to 35 minutes, or until a toothpick inserted in the center comes out clean.
  7. Let the cakes cool in the pans for 10 minutes, then remove from pans to cool completely on wire racks.

- **Frosting Suggestion:**
  - You can frost the cake with chocolate buttercream or cream cheese frosting for added flavor.

### Relevant Tags:
- Baking
- Desserts
- Chocolate

Enjoy your baking! If you have any other questions or need more recipes, feel free to ask!

## RAG System

In [36]:
# Build the function for Retrieval-Augmented Generation (RAG)
def rag_system(query, index, metadata, system_prompt, k = 5):
  # Retrieval System: Retrieve relevant results based on the query
  results = query_embeddings(query, index, metadata, k)

  # Content Merge: Combine the retrieved content into a single string
  combined_content = combined_retrived_content(results)

  # Generation: Generate a response based on the query and combined content
  response = generate_response(query, combined_content, system_prompt)

  # Return the generated response
  return response

In [37]:
# Test the rag system
query1 = "How to make the best chocolate cake?"
response = rag_system(query1, index, metadata, system_prompt3)
get_gpt_response()

The query embedding is [-0.002604733919724822, -0.03968388959765434, -0.013297276571393013, 0.007617205381393433, -0.019327564164996147, 0.022370068356394768, 0.00810969714075327, -0.005398256704211235, -0.006457113660871983, 0.037473149597644806, -0.0003669747384265065, 0.007113769184798002, -0.05170068517327309, -0.007600788958370686, 0.0003343129646964371, -0.04031865671277046, 0.021166199818253517, -0.004692351911216974, -0.004487146623432636, -0.015858232975006104, 0.015004580840468407, -0.02479969523847103, -0.011732247658073902, 0.03399287164211273, 0.0030698650516569614, 0.006758080795407295, 0.022610843181610107, -0.0006980386096984148, -0.036904048174619675, 0.05677882209420204, 0.04460880532860756, 0.0018906210316345096, -0.016208449378609657, -0.014720030128955841, -0.02911173366010189, 0.016919827088713646, -0.004768961574882269, -0.001930293976329267, 0.03475897014141083, 0.014402646571397781, -0.005751208867877722, 5.8526144130155444e-05, 0.002621150342747569, 0.01079104

To make the best chocolate cake, here's a classic recipe you can follow:

### Best Chocolate Cake Recipe

#### Ingredients:
- **Dry Ingredients:**
  - 1 ¾ cups all-purpose flour
  - 1 ¾ cups granulated sugar
  - ¾ cup unsweetened cocoa powder
  - 1 ½ teaspoons baking powder
  - 1 ½ teaspoons baking soda
  - 1 teaspoon salt

- **Wet Ingredients:**
  - 2 large eggs
  - 1 cup whole milk
  - ½ cup vegetable oil
  - 2 teaspoons vanilla extract
  - 1 cup boiling water

#### Instructions:
1. **Preheat the Oven:**
   - Preheat your oven to 350°F (175°C). Grease and flour two 9-inch round cake pans.

2. **Mix Dry Ingredients:**
   - In a large mixing bowl, combine the flour, sugar, cocoa powder, baking powder, baking soda, and salt. Whisk together until well combined.

3. **Add Wet Ingredients:**
   - Add the eggs, milk, vegetable oil, and vanilla extract to the dry ingredients. Beat on medium speed for about 2 minutes until well blended.

4. **Incorporate Boiling Water:**
   - Carefully stir in the boiling water (the batter will be thin). Mix until smooth.

5. **Bake:**
   - Pour the batter evenly into the prepared cake pans. Bake for 30-35 minutes, or until a toothpick inserted in the center comes out clean.

6. **Cool:**
   - Allow the cakes to cool in the pans for 10 minutes, then remove from pans and transfer to wire racks to cool completely.

7. **Frosting (Optional):**
   - You can frost the cake with your favorite chocolate frosting once it has cooled completely.

### Tips for the Best Chocolate Cake:
- Use high-quality cocoa powder for a richer flavor.
- Make sure all ingredients are at room temperature before mixing.
- Do not overmix the batter; mix just until combined.
- For added moisture, you can substitute some of the water with brewed coffee.

Enjoy your delicious homemade chocolate cake! If you have any specific preferences or variations in mind, feel free to ask!

In [38]:
# Test with a different query
query2 = "I want something vegan"
response = rag_system(query2, index, metadata, system_prompt3)
get_gpt_response()

The query embedding is [-0.034548308700323105, -0.026619188487529755, -0.017104245722293854, 0.03446335345506668, -0.01730247214436531, -0.008516724221408367, 5.26128314959351e-06, -0.018930774182081223, -0.033670444041490555, 0.023518336936831474, 0.003238904057070613, -0.012955616228282452, 0.0179679524153471, -0.007178685627877712, 0.036842089146375656, -0.008821146562695503, 0.009663615375757217, -0.01138395071029663, 0.007461868692189455, -0.018406886607408524, -0.023546654731035233, 0.00599993672221899, 0.037436775863170624, -0.0069061219692230225, 0.007702574133872986, 0.008198143914341927, -0.022074105218052864, -0.012240579351782799, 0.002099093049764633, 0.01846352219581604, 0.010208742693066597, 0.024481158703565598, -0.0050618937239050865, 0.05527729541063309, -0.013698970898985863, -0.003830048255622387, 0.038739416748285294, -0.0019114842871204019, 0.015617535449564457, 0.026930689811706543, -0.04777294769883156, 0.022003307938575745, -0.024226294830441475, 0.001365472329

I can't view the image directly, but I can help you with vegan recipes if you provide some details or specific ingredients you're interested in. Here are a few simple vegan recipe ideas:

### Vegan Chocolate Cake
- **Ingredients:**
  - 1 ½ cups all-purpose flour
  - 1 cup granulated sugar
  - 1/3 cup cocoa powder
  - 1 teaspoon baking soda
  - 1/2 teaspoon salt
  - 1 cup almond milk (or any plant-based milk)
  - 1/3 cup vegetable oil
  - 1 tablespoon apple cider vinegar
  - 1 teaspoon vanilla extract

- **Instructions:**
  1. Preheat the oven to 350°F (175°C) and grease a cake pan.
  2. In a bowl, mix the dry ingredients: flour, sugar, cocoa powder, baking soda, and salt.
  3. In another bowl, combine the wet ingredients: almond milk, oil, vinegar, and vanilla.
  4. Pour the wet mixture into the dry ingredients and mix until just combined.
  5. Pour the batter into the prepared pan and bake for 30-35 minutes.
  6. Let cool before serving.

### Vegan Banana Bread
- **Ingredients:**
  - 3 ripe bananas, mashed
  - 1/3 cup melted coconut oil
  - 1 teaspoon vanilla extract
  - 1 teaspoon baking soda
  - Pinch of salt
  - 3/4 cup sugar
  - 1 ½ cups all-purpose flour

- **Instructions:**
  1. Preheat the oven to 350°F (175°C) and grease a loaf pan.
  2. In a mixing bowl, combine the mashed bananas and melted coconut oil.
  3. Stir in the vanilla, baking soda, and salt.
  4. Mix in the sugar and flour until just combined.
  5. Pour the batter into the prepared loaf pan and bake for 50-60 minutes.
  6. Let cool before slicing.

If you have specific ingredients or types of dishes in mind, let me know, and I can provide more tailored vegan recipes!